In [43]:
# Candidato: Lúcio Leandro Cruz de Oliveira, Email: 95lucioleandro@gmail.com
# Obs: Todas os códigos estão comentados, caso não esteja, é porque já foram comentados acima.
# Obs2: Os valores referentes a dinheiro estão com os decimais separados por ponto e sem separação de milhar.
# Obs3: Escolhi fazer a parte de ETL no Jupyter devido a facilidade de execução, porém, poderia usar qualquer outra ferramenta.

# Importando as bibliotecas que serão utilizadas
import pandas as pd
import datetime as dt

# Lendo as informações da empresa - XLSX

In [68]:
# Lendo o XLSX com as informações da empresa
empresa = pd.ExcelFile('Dimensoes_DadosModelagem.xlsx')

# Lendo separadamente cada aba da planilha
empresa_cliente = pd.read_excel(empresa, sheet_name='Clientes')
empresa_funcionarios = pd.read_excel(empresa, sheet_name='Funcionarios')
empresa_produtos = pd.read_excel(empresa, sheet_name='Produtos')
empresa_categorias = pd.read_excel(empresa, sheet_name='Categoria')

# Planilha com os produtos vendidos - CSV

In [69]:
# Lendo o CSV com os detalhes dos produtos
produtos = pd.read_csv('FatoDetalhes_DadosModelagem.csv', sep=";")

# Alterando o nome apenas para meu melhor entendimento
produtos = produtos.rename(columns={'ValorLiquido':'Lucro'})

# Convertendo as strings para float, sendo preciso tirar a virgula primeiro
produtos['Valor'] = produtos['Valor'].str.replace(',', '.').astype(float)
produtos['Desconto'] = produtos['Desconto'].str.replace(',', '.').astype(float)
produtos['Custo'] = produtos['Custo'].str.replace(',', '.').astype(float)
produtos['Lucro'] = produtos['Lucro'].str.replace(',', '.').astype(float)

# Arredondando para 2 casas decimais
produtos[['Valor', 'Desconto', 'Custo', 'Lucro']] = round(produtos[['Valor', 'Desconto', 'Custo', 'Lucro']], 2)

# Acrescentando o nome dos produtos e categorias, para ficar melhor de visualizar
produtos = pd.merge(produtos, empresa_produtos[['ProdutoID','Produto']], how='inner', on='ProdutoID')
empresa_produtos = pd.merge(empresa_produtos, empresa_categorias[['CategoriaID','Categoria']], how='inner', on='CategoriaID')
produtos = pd.merge(produtos, empresa_produtos[['ProdutoID','Categoria']], how='inner', on='ProdutoID')

# Mostrando a planilha de forma ordenada por produtos
display(produtos.sort_values(by=['ProdutoID']).head(5))

,CupomID,ProdutoID,Quantidade,Valor,Desconto,Custo,Lucro,Produto,Categoria
975,10729,1,50,1026.00,0.00,831.06,194.94,Lenin Jeansshorts,Mens Clothes
952,10406,1,10,197.60,0.00,163.61,33.99,Lenin Jeansshorts,Mens Clothes
953,11025,1,10,215.05,19.55,180.17,34.88,Lenin Jeansshorts,Mens Clothes
954,10911,1,10,219.30,0.00,197.02,22.28,Lenin Jeansshorts,Mens Clothes
955,10576,1,10,219.40,0.00,177.71,41.69,Lenin Jeansshorts,Mens Clothes


In [56]:
# Verificando se está tudo formatado corretamente
produtos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2172 entries, 0 to 2171
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CupomID     2172 non-null   int64  
 1   ProdutoID   2172 non-null   int64  
 2   Quantidade  2172 non-null   int64  
 3   Valor       2172 non-null   float64
 4   Desconto    2172 non-null   float64
 5   Custo       2172 non-null   float64
 6   Lucro       2172 non-null   float64
 7   Produto     2172 non-null   object 
dtypes: float64(4), int64(3), object(1)
memory usage: 152.7+ KB


### Separando apenas para fins de organização, todos os dados já poderiam ser retirados da tabela acima

In [74]:
# Filtrando os dados de quais produtos foram vendidos em cada compra - Gerando o CSV 'Compra_Produto'
cupom_produto = produtos.loc[:,['CupomID','ProdutoID','Produto', 'Categoria', 'Quantidade', 'Valor', 'Custo','Lucro']]

# Organizando de acordo com a prioridade de agrupamento
cupom_produto = cupom_produto.groupby(['CupomID','ProdutoID', 'Produto','Categoria', 'Quantidade', 'Valor', 'Custo']).sum(['Lucro'])

#Salvando em um CSV
cupom_produto.to_csv('.\DadosFiltrados\Compra_Produto.csv', sep=';')

# Mostrando no jupyter
display(cupom_produto.head(10))

Lucro
CupomID ProdutoID Produto             Categoria       Quantidade Valor   Custo           
10248   11        Desperado Jeans     Mens Clothes    12         343.44  284.37     59.07
        42        Balett Shoes        Womens Footwear 10         140.40  116.25     24.15
        72        Rossi Shorts        Bath Clothes    5          63.55   52.62      10.93
10249   14        Kool Sunglasses     Childrens wear  9          205.20  169.91     35.29
        51        Snake Boots         Mens Footwear   40         4048.00 3642.67   405.33
10250   41        Duck Shirt          Babywear        10         95.90   79.41      16.49
        51        Snake Boots         Mens Footwear   35         3931.22 2830.48  1100.74
        65        Stretch oui-pants   Womens wear     15         163.36  117.62     45.74
10251   22        Ravellis Träskor    Womens Footwear 6          21.36   16.84       4.52
        57        Burned Rubber Shoes Womens Footwear 15         332.64  288.14     44.50

In [58]:
# Filtrando para saber qual produto é o mais vendido - Gerando o CSV 'Produto_Quantidade'
produto_quantidade = produtos.loc[:,['CupomID','ProdutoID','Produto','Quantidade', 'Valor','Lucro']]
produto_quantidade = produto_quantidade.groupby(['ProdutoID','Produto','Quantidade', 'Valor', 'CupomID']).sum(['Lucro'])
produto_quantidade.to_csv('.\DadosFiltrados\Produto_Quantidade.csv', sep=';')
display(produto_quantidade.head(10))

Lucro
ProdutoID Produto           Quantidade Valor  CupomID       
1         Lenin Jeansshorts 2          38.00  11005     4.48
                            3          55.50  10609    10.54
                            4          76.00  11003     8.97
                                       102.60 10838    30.21
                            5          121.86 10700    39.60
                            6          118.50 10611    22.52
                            8          144.16 10752    27.39
                                       175.52 11006    20.71
                                       179.68 10526    50.31
                            10         195.50 11035    23.07

# Planilha com as vendas realizadas - TXT

In [71]:
# Lendo o TXT com os detalhes das vendas
vendas = pd.read_csv('FatoCabecalho_DadosModelagem.txt', sep="	")

# Formatando adequatamente as datas
vendas[['Data','DataEntrega']] = vendas[['Data','DataEntrega']].apply(pd.to_datetime, format='%d/%m/%Y')

# Verificando o tempo de entrega de cada compra
vendas['TempoEntrega'] = vendas['DataEntrega'] - vendas['Data']

# Separando o ano de venda
vendas['AnoVenda'] = vendas['Data'].dt.year

# Separando por trimestre
vendas['TrimestreVenda'] = vendas['Data'].dt.quarter

# Acrescentando os nomes do clientes e funcionarios, para ficar melhor de visualizar
vendas = pd.merge(vendas, empresa_funcionarios[['FuncionarioID','NomeFuncionario']], how='inner', on='FuncionarioID')
vendas = pd.merge(vendas, empresa_cliente[['ClienteID','Cliente', 'Pais']], how='inner', on='ClienteID')

# Mostrando a planilha final
display(vendas.head(5))

,Data,ClienteID,FuncionarioID,ValorFrete,CupomID,EmpresaFrete,DataEntrega,TempoEntrega,AnoVenda,TrimestreVenda,NomeFuncionario,Cliente,Pais
0,2018-03-22,78,11,"12,45",11081,Freight Express,2018-04-05,14 days,2018,1,Rock Rollman,TTT-The Ticky Tie,USA
1,2017-07-02,80,11,"20,53",10982,Freight Express,2017-07-04,2 days,2017,3,Rock Rollman,Don Balón,Mexico
2,2015-01-11,80,11,"22,95",10276,Freight Express,2015-01-19,8 days,2015,1,Rock Rollman,Don Balón,Mexico
3,2017-06-01,80,11,"24,52",10915,Freight Express,2017-06-15,14 days,2017,2,Rock Rollman,Don Balón,Mexico
4,2015-10-30,80,11,"27,16",10518,Freight Express,2015-10-31,1 days,2015,4,Rock Rollman,Don Balón,Mexico


In [60]:
# Verificando se está tudo formatado corretamente
vendas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836 entries, 0 to 835
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   Data             836 non-null    datetime64[ns] 
 1   ClienteID        836 non-null    int64          
 2   FuncionarioID    836 non-null    int64          
 3   ValorFrete       836 non-null    object         
 4   CupomID          836 non-null    int64          
 5   EmpresaFrete     836 non-null    object         
 6   DataEntrega      836 non-null    datetime64[ns] 
 7   TempoEntrega     836 non-null    timedelta64[ns]
 8   AnoVenda         836 non-null    int64          
 9   TrimestreVenda   836 non-null    int64          
 10  NomeFuncionario  836 non-null    object         
 11  Cliente          836 non-null    object         
dtypes: datetime64[ns](2), int64(5), object(4), timedelta64[ns](1)
memory usage: 84.9+ KB


### Separando apenas para fins de organização, todos os dados já poderiam ser retirados da tabela acima

In [61]:
# Separando as compras por data para saber qual a melhor época de vendas - Gerando o CSV 'Vendas_Data'
vendas_data = vendas.loc[:,['AnoVenda','TrimestreVenda', 'Data','CupomID']]

# Agrupando
vendas_data = vendas_data.groupby(['AnoVenda', 'TrimestreVenda', 'Data', 'CupomID']).sum()

# Salvando em CSV
vendas_data.to_csv('.\DadosFiltrados\Vendas_Data.csv', sep=';')

# Mostrando no Jupyter
display(vendas_data.head(10))

Empty DataFrame
Columns: []
Index: [(2014, 3, 2014-07-30 00:00:00, 10337), (2014, 3, 2014-08-28 00:00:00, 10262), (2014, 3, 2014-09-08 00:00:00, 10374), (2014, 3, 2014-09-22 00:00:00, 10357), (2014, 3, 2014-09-24 00:00:00, 10286), (2014, 3, 2014-09-25 00:00:00, 10287), (2014, 4, 2014-10-11 00:00:00, 10296), (2014, 4, 2014-11-03 00:00:00, 10268), (2014, 4, 2014-11-15 00:00:00, 10278), (2014, 4, 2014-11-30 00:00:00, 10291)]

In [64]:
# Separando por funcionário para saber quais vendem mais e quais vendem menos - Gerando o CSV 'Vendas_Funcionarios'
vendas_funcionario = vendas.loc[:,['AnoVenda', 'FuncionarioID','NomeFuncionario','CupomID', 'TrimestreVenda']]
vendas_funcionario = vendas_funcionario.groupby(['FuncionarioID','NomeFuncionario','AnoVenda', 'TrimestreVenda', 'CupomID']).sum()
vendas_funcionario.to_csv('.\DadosFiltrados\Vendas_Funcionarios.csv', sep=';')
display(vendas_funcionario.head(10))

Empty DataFrame
Columns: []
Index: [(2, Eli Preston, 2014, 3, 10262), (2, Eli Preston, 2014, 3, 10287), (2, Eli Preston, 2014, 3, 10357), (2, Eli Preston, 2014, 4, 10278), (2, Eli Preston, 2014, 4, 10291), (2, Eli Preston, 2014, 4, 10296), (2, Eli Preston, 2014, 4, 10298), (2, Eli Preston, 2015, 1, 10271), (2, Eli Preston, 2015, 1, 10344), (2, Eli Preston, 2015, 1, 10347)]

In [72]:
# Separando os clientes para saber qual compra mais - Gerando o CSV 'Vendas_Cliente'
vendas_cliente = vendas.loc[:,['ClienteID', 'Cliente','Pais', 'AnoVenda','CupomID', 'TrimestreVenda']]
vendas_cliente = vendas_cliente.groupby(['ClienteID','Cliente','Pais','AnoVenda', 'TrimestreVenda', 'CupomID']).sum()
vendas_cliente.to_csv('.\DadosFiltrados\Vendas_Cliente.csv', sep=';')
display(vendas_cliente.head(10))

Empty DataFrame
Columns: []
Index: [(1, Eintrach GS, Germany, 2015, 1, 10338), (1, Eintrach GS, Germany, 2015, 4, 10526), (1, Eintrach GS, Germany, 2016, 3, 10692), (1, Eintrach GS, Germany, 2016, 3, 10750), (1, Eintrach GS, Germany, 2016, 3, 10757), (1, Eintrach GS, Germany, 2016, 4, 10289), (1, Eintrach GS, Germany, 2016, 4, 10510), (1, Eintrach GS, Germany, 2016, 4, 10621), (1, Eintrach GS, Germany, 2016, 4, 10627), (1, Eintrach GS, Germany, 2017, 1, 10251)]

In [67]:
# Separando por empresa de frete para saber qual entrega mais rápido e qual tem mais demanda - Gerando o CSV 'Frete_Empresa'
frete_empresa = vendas.loc[:,['Data','CupomID', 'EmpresaFrete', 'DataEntrega','TempoEntrega']]
frete_empresa = frete_empresa.groupby(['EmpresaFrete','Data', 'DataEntrega', 'TempoEntrega', 'CupomID']).sum()
frete_empresa.to_csv('.\DadosFiltrados\Frete_Empresa.csv', sep=';')
display(frete_empresa.head(10))

Empty DataFrame
Columns: []
Index: [(Freight Express, 2015-01-11 00:00:00, 2015-01-19 00:00:00, 8 days 00:00:00, 10276), (Freight Express, 2015-01-31 00:00:00, 2015-02-06 00:00:00, 6 days 00:00:00, 10339), (Freight Express, 2015-02-14 00:00:00, 2015-03-01 00:00:00, 15 days 00:00:00, 10351), (Freight Express, 2015-03-06 00:00:00, 2015-03-15 00:00:00, 9 days 00:00:00, 10369), (Freight Express, 2015-03-15 00:00:00, 2015-04-01 00:00:00, 17 days 00:00:00, 10377), (Freight Express, 2015-03-21 00:00:00, 2015-04-06 00:00:00, 16 days 00:00:00, 10329), (Freight Express, 2015-04-15 00:00:00, 2015-04-21 00:00:00, 6 days 00:00:00, 10564), (Freight Express, 2015-04-15 00:00:00, 2015-04-27 00:00:00, 12 days 00:00:00, 10410), (Freight Express, 2015-06-05 00:00:00, 2015-06-19 00:00:00, 14 days 00:00:00, 10431), (Freight Express, 2015-06-15 00:00:00, 2015-06-21 00:00:00, 6 days 00:00:00, 10441)]